<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/adadelta_and_SGD_hyper%2Bsingle_sparsity_selectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(1234)
np.random.seed(1234)

In [4]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))


class_inds = [torch.where(mnist_trainset.targets == class_idx)[0]
              for class_idx in mnist_trainset.class_to_idx.values()]


train_dataloaders = [
                     DataLoader(dataset=Subset(mnist_trainset, inds),
                                batch_size=50,
                                shuffle=True,
                                drop_last=False
                     )
                     for inds in class_inds
]

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

Training dataset size:  60000
Testing dataset size:  10000


In [5]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [6]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [7]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [8]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)

    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [9]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [10]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    # modify this section for later use 
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    # ADADELTA
    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0, rho=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1, momentum=0.9)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity_avg, selectivity_list_avg, selectivity_list_std = selectivity_trainer(optimizer=my_optimizer, model=my_model)
    # ************* modify this section for later use *************
    file_saver = open(f"combine_sparsity_selectivity_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity_avg)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp combine_sparsity_selectivity_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp combine_sparsity_selectivity_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp combine_sparsity_selectivity_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp combine_sparsity_selectivity_Adam.txt /content/drive/MyDrive
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [11]:
no_epochs = 30
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    selectivity_avg_list = list()
    selectivity_std_list = list()

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()
        iterators = list(map(iter, train_dataloaders))   
        while iterators:    # This part is same as for loop 
            iterator = np.random.choice(iterators)
            try:
                image, label = next(iterator)   
                image, label = image.to(device), label.to(device)

                optimizer.zero_grad()

                pred = model(image)

                loss = criterion(pred, label)

                loss.backward()
                optimizer.step()
                
            except StopIteration:
                iterators.remove(iterator)


        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1
            
            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])

        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** #
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('Epoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))
    
    # ***************** sparsity calculation ***************** #
    sparsity_list12 = sparsity_calculator(final_spareness_12)

    average_sparsity = list()
    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)

    print(selectivity_avg_list)
    print(selectivity_std_list)
    return test_acc, sparsity_list12, selectivity_avg_list, selectivity_std_list

In [12]:
model_factory('Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 1.0
    weight_decay: 0
)
Epoch: 1/30, Train Loss: 0.00000000, Test Loss: 1.12535267, Test Accuracy: 0.66500000
Epoch: 2/30, Train Loss: 0.00000000, Test Loss: 0.92132770, Test Accuracy: 0.74100000
Epoch: 3/30, Train Loss: 0.00000000, Test Loss: 0.41403226, Test Accuracy: 0.87170000
Epoch: 4/30, Train Loss: 0.00000000, Test Loss: 0.49222100, Test Accuracy: 0.84650000
Epoch: 5/30, Train Loss: 0.00000000, Test Loss: 0.26893806, Test Accuracy: 0.91460000
Epoch: 6/30, Train Loss: 0.00000000, Test Loss: 0.27824106, Test Accuracy: 0.91500000
Epoch: 7/30, Train Loss: 0.00000000, Test Loss: 0.32562472, Test Accuracy: 0.89920000
Epoch: 8/30, Train Loss: 0.00000000, Test Loss: 0.26091860, Test Accuracy: 0.91830000
Epoch: 9/30, Train Loss: 0.000

In [13]:
model_factory('SGD')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
Epoch: 1/30, Train Loss: 0.00000000, Test Loss: 1.81769768, Test Accuracy: 0.56590000
Epoch: 2/30, Train Loss: 0.00000000, Test Loss: 0.54762083, Test Accuracy: 0.83940000
Epoch: 3/30, Train Loss: 0.00000000, Test Loss: 0.33984126, Test Accuracy: 0.90280000
Epoch: 4/30, Train Loss: 0.00000000, Test Loss: 0.43677573, Test Accuracy: 0.86430000
Epoch: 5/30, Train Loss: 0.00000000, Test Loss: 0.32052384, Test Accuracy: 0.90240000
Epoch: 6/30, Train Loss: 0.00000000, Test Loss: 0.37957823, Test Accuracy: 0.89210000
Epoch: 7/30, Train Loss: 0.00000000, Test Loss: 0.33457515, Test Accuracy: 0.89640000
Epoch: 8/30, Train Loss: 0.00000000, Test Loss: 0.23201332, Test Accuracy: 0.93170000
Epoch: 9